In [1]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

from keras.models import Sequential
from keras.layers import Dense, Dropout

Using TensorFlow backend.


## Data Analysis

In [8]:
df_train, df_val = train_test_split(pd.read_csv('train.csv').sample(5000))

In [9]:
pd.merge(pd.DataFrame(df_train.iloc[:,2:].sum() / len(df_train) * 100, columns=['% train']),
         pd.DataFrame(df_val.iloc[:,2:].sum() / len(df_val) * 100, columns=['% val']),
         right_index=True,
         left_index=True)

,% train,% val
toxic,9.840000,10.96
severe_toxic,0.746667,1.12
obscene,5.546667,5.84
threat,0.266667,0.16
insult,4.826667,4.88
identity_hate,0.693333,1.12


## Training

In [10]:
tfidf = TfidfVectorizer()

In [11]:
X_train = tfidf.fit_transform(df_train['comment_text']).todense()
y_train = df_train.iloc[:,2:].values

X_val = tfidf.transform(df_val['comment_text']).todense()
y_val = df_val.iloc[:,2:].values

In [12]:
print X_train.shape, y_train.shape
print X_val.shape, y_val.shape

(3750, 20222) (3750, 6)
(1250, 20222) (1250, 6)


In [13]:
model = Sequential()
model.add(Dense(1024,
                activation='relu',
                input_shape=((X_train.shape[1],))))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=6,
                activation='sigmoid'))
model.compile('adam', 'binary_crossentropy')

model.fit(X_train,
          y_train,
          validation_data=(X_val, y_val),
          epochs=1)

Train on 3750 samples, validate on 1250 samples
Epoch 1/1
3750/3750 [==============================] - 91s - loss: 0.1678 - val_loss: 0.1025


In [14]:
model.predict_classes(X_val)

1248/1250 [============================>.] - ETA: 0s

array([0, 0, 0, ..., 0, 0, 0])

## Test

In [16]:
df_test = pd.read_csv('test.csv')
X_test = tfidf.transform(df_test['comment_text']).todense()
y_pred = model.predict_proba(X_test)

 23296/153164 [===>..........................] - ETA: 304s

KeyboardInterrupt: 

In [ ]:
submission_df = pd.read_csv('sample_submission.csv')
submission_df.iloc[:,1:] = y_pred

In [ ]:
submission_df.head()